
# Full-heart mechanics
This example shows you how to consume a preprocessed full heart model and
set it up for the main mechanical simulation. This examples demonstrates how
you can load a pre-computed heart model, compute the fiber direction, compute the
stress free configuration, and finally simulate the mechanical model.


# Example setup
before computing the fiber orientation, and stress free configuration we
need to load the required modules, load a heart model and configure the
mechanical simulator.

## Perform the required imports
Import the required modules and set relevant paths, including that of the working
directory, model, and ls-dyna executable.



In [ ]:
import os

import ansys.heart.preprocessor.models as models
from ansys.heart.simulator.simulator import DynaSettings, MechanicsSimulator

# accept dpf license aggrement
# https://dpf.docs.pyansys.com/version/stable/getting_started/licensing.html#ref-licensing
os.environ["ANSYS_DPF_ACCEPT_LA"] = "Y"

# set working directory and path to model.
workdir = os.path.join("pyansys-heart", "downloads", "Rodero2021", "01", "FullHeart")


path_to_model = os.path.join(workdir, "heart_model.vtu")

# load four chamber heart model.
model: models.FullHeart = models.FullHeart(models.ModelInfo(work_directory=workdir))
model.load_model_from_mesh(path_to_model, path_to_model.replace(".vtu", ".partinfo.json"))
model._extract_apex()
model.compute_left_ventricle_anatomy_axis()
model.compute_left_ventricle_aha17()

# set base working directory
model.info.workdir = str(workdir)

## Instantiate the simulator object
instantiate your DynaSettings and Simulator objects.
Change options where necessary. Note that you may need to configure your environment
variables if you choose to use a `mpi` version of LS-DYNA.



In [ ]:
# instantiate dyna settings object
lsdyna_path = "lsdyna_intelmpi"
# instantiate dyna settings object
dyna_settings = DynaSettings(
    lsdyna_path=lsdyna_path,
    dynatype="intelmpi",
    num_cpus=8,
)


# instantiate simulator object
simulator = MechanicsSimulator(
    model=model,
    dyna_settings=dyna_settings,
    simulation_directory=os.path.join(workdir, "simulation-mechanics"),
)

## Load simulation settings
Here we load the default settings.



In [ ]:
simulator.settings.load_defaults()

## Compute the fiber orientation
Compute fiber orientation and plot the computed fibers on the entire model.



In [ ]:
simulator.compute_fibers()

# # Plot the resulting fiber orientation
simulator.model.plot_fibers(n_seed_points=2000)

<img src="file://_static/images/full_heart_rodero_01_fibers.png" width="400pt" align="center">



## Compute the stress free configuration
Compute the stress free configuration. That is, when imaged under diastole
we need to approximate the initial stress at `t=0`. The stress free configuration
is computed through Rausch' method.



In [ ]:
simulator.compute_stress_free_configuration()

## Start main simulation
Start the main mechanical simulation. This uses the previously computed fiber orientation
and stress free configuration and runs the final LS-DYNA heart model.



In [ ]:
simulator.simulate()